*PART 4: Second and Third Steps*

***

# What Will Happen in This Part?

We will be creating 3 different models to predict `grade`, `sub_grade` and `int_rate`.

# Data and Module Import

In [25]:
import pandas as pd
import numpy as np
import warnings
from utilities import *
from sklearn.preprocessing import StandardScaler, MinMaxScaler 
from category_encoders import OneHotEncoder, OrdinalEncoder, PolynomialEncoder, HelmertEncoder
from sklearn.compose import ColumnTransformer
from imblearn.pipeline import Pipeline, make_pipeline
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, make_scorer, fbeta_score, f1_score
from sklearn.model_selection import cross_validate, GridSearchCV, StratifiedKFold, train_test_split
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import precision_recall_curve, mean_absolute_error
from sklearn.utils import resample
import optuna
import pickle
from sklearn.utils import class_weight

warnings.filterwarnings('ignore')

df = pd.read_csv('./processed_data.csv', index_col=0)

df.head()

,loan_amnt,term,emp_title,emp_length,home_ownership,annual_inc,purpose,addr_state,dti,inq_last_6mths,...,pub_rec_bankruptcies,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,disbursement_method,grade,sub_grade,loan_status,int_rate
0,3600.0,36 months,other,10+ years,MORTGAGE,55000.0,debt_consolidation,PA,5.91,1.0,...,0.0,178050.0,7746.0,2400.0,13734.0,Cash,C,C4,1,13.99
2,20000.0,60 months,driver,10+ years,MORTGAGE,63000.0,home_improvement,IL,10.78,0.0,...,0.0,218418.0,18696.0,6200.0,14877.0,Cash,B,B4,1,10.78
4,10400.0,60 months,other,3 years,MORTGAGE,104433.0,major_purchase,PA,25.37,3.0,...,0.0,439570.0,95768.0,20300.0,88097.0,Cash,F,F1,1,22.45
5,11950.0,36 months,other,4 years,RENT,34000.0,debt_consolidation,GA,10.20,0.0,...,0.0,16900.0,12798.0,9400.0,4000.0,Cash,C,C3,1,13.44
7,20000.0,36 months,driver,10+ years,MORTGAGE,85000.0,major_purchase,SC,17.61,0.0,...,0.0,193390.0,27937.0,14500.0,36144.0,Cash,B,B1,1,8.49


***

# 5. Predicting Grade

We will be using Random Forest Classifiers for this task, as it was the best model for classifying `loan_status` and the data hasn't changed.

## 5.1. Data Preparation

First, let's cut off our target feature and remove other target features.

In [2]:
X = df.drop(columns=['grade', 'sub_grade', 'int_rate'])
y = df['grade']

Sampling down the dataset for less resourcefully demanding model.

In [3]:
X, y = resample(X, y, n_samples=len(X) // 10, random_state=42)

Train, test split.

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42, stratify=y)

Saving column names as either numerical or categorical features.

In [5]:
categorical_features = X.dtypes[df.dtypes != np.number].index
numerical_features = X.dtypes[df.dtypes == np.number].index

Calculating class weights for `grade`.

In [6]:
y_unique = np.unique(y_train.to_list())

class_weights = class_weight.compute_class_weight('balanced', classes=y_unique, y=y_train)

class_weights = {y_unique[i]: class_weights[i] for i in range(len(y_unique))}

### Model Tuning

In [7]:
encoder = OneHotEncoder()

column_transformer = ColumnTransformer([
    ('cat', encoder, categorical_features)
])

def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 10, 200, log=True)
    max_depth = trial.suggest_int('max_depth', 2, 32)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)

    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        class_weight=class_weights,
        random_state=42
    )

    pipe = make_pipeline(column_transformer, TruncatedSVD(), model)

    pipe.fit(X_train, y_train)

    y_pred = pipe.predict(X_test)
    
    accuracy = accuracy_score(y_pred, y_test)
    
    return accuracy

In [8]:
study = optuna.create_study(direction='maximize')

study.optimize(objective, n_trials=20, show_progress_bar=True)

[I 2023-08-31 19:25:19,883] A new study created in memory with name: no-name-bd9e5e31-4c13-4d13-9570-355c704d8119
Best trial: 0. Best value: 0.230004:   5%|▌         | 1/20 [00:01<00:33,  1.76s/it]

[I 2023-08-31 19:25:21,644] Trial 0 finished with value: 0.230004366958917 and parameters: {'n_estimators': 19, 'max_depth': 9, 'min_samples_split': 7, 'min_samples_leaf': 7}. Best is trial 0 with value: 0.230004366958917.


Best trial: 0. Best value: 0.230004:  10%|█         | 2/20 [00:04<00:39,  2.17s/it]

[I 2023-08-31 19:25:24,098] Trial 1 finished with value: 0.2186502737747321 and parameters: {'n_estimators': 37, 'max_depth': 8, 'min_samples_split': 7, 'min_samples_leaf': 2}. Best is trial 0 with value: 0.230004366958917.


Best trial: 2. Best value: 0.282072:  15%|█▌        | 3/20 [00:07<00:44,  2.62s/it]

...

[I 2023-08-31 19:26:16,963] Trial 17 finished with value: 0.2631596627364036 and parameters: {'n_estimators': 23, 'max_depth': 32, 'min_samples_split': 8, 'min_samples_leaf': 9}. Best is trial 4 with value: 0.29174644764688096.


Best trial: 4. Best value: 0.291746:  95%|█████████▌| 19/20 [01:02<00:03,  3.62s/it]

[I 2023-08-31 19:26:22,636] Trial 18 finished with value: 0.2774362591958077 and parameters: {'n_estimators': 67, 'max_depth': 21, 'min_samples_split': 5, 'min_samples_leaf': 5}. Best is trial 4 with value: 0.29174644764688096.


Best trial: 4. Best value: 0.291746: 100%|██████████| 20/20 [01:10<00:00,  3.53s/it]

[I 2023-08-31 19:26:30,465] Trial 19 finished with value: 0.28438980147132925 and parameters: {'n_estimators': 91, 'max_depth': 26, 'min_samples_split': 3, 'min_samples_leaf': 3}. Best is trial 4 with value: 0.29174644764688096.


Our best model for predicting grade only has an accuracy of 29%.

### Saving the Model

In [9]:
model = RandomForestClassifier(**study.best_params)

pickle.dump(model, open('./model/app/model_grade.pkl', 'wb'))

# 6. Predicting Sub Grade

## 6.1. Data Preparation

Cutting of the target feature.

In [10]:
X = df.drop(columns=['sub_grade', 'int_rate'])
y = df['sub_grade']

Sampling down the data.

In [11]:
X, y = resample(X, y, n_samples=len(X) // 10, random_state=42)

Train, test split.

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42, stratify=y)

Saving feature names.

In [13]:
categorical_features = X.dtypes[df.dtypes != np.number].index
numerical_features = X.dtypes[df.dtypes == np.number].index

Calculating class weights.

In [14]:
y_unique = np.unique(y_train.to_list())

class_weights = class_weight.compute_class_weight('balanced', classes=y_unique, y=y_train)

class_weights = {y_unique[i]: class_weights[i] for i in range(len(y_unique))}

### Model Tuning

In [15]:
encoder = OneHotEncoder()

column_transformer = ColumnTransformer([
    ('cat', encoder, categorical_features)
])

def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 10, 200, log=True)
    max_depth = trial.suggest_int('max_depth', 2, 32)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)

    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        class_weight=class_weights,
        random_state=42
    )

    pipe = make_pipeline(column_transformer, TruncatedSVD(), model)

    pipe.fit(X_train, y_train)

    y_pred = pipe.predict(X_test)
    
    accuracy = accuracy_score(y_pred, y_test)
    
    return accuracy

In [16]:
study = optuna.create_study(direction='maximize')

study.optimize(objective, n_trials=20, show_progress_bar=True)

[I 2023-08-31 19:26:30,783] A new study created in memory with name: no-name-895a01b2-0e4c-4d98-94d5-f4b753bbde10
Best trial: 0. Best value: 0.0991972:   5%|▌         | 1/20 [00:03<00:57,  3.01s/it]

[I 2023-08-31 19:26:33,790] Trial 0 finished with value: 0.09919715139910645 and parameters: {'n_estimators': 34, 'max_depth': 8, 'min_samples_split': 6, 'min_samples_leaf': 8}. Best is trial 0 with value: 0.09919715139910645.


Best trial: 1. Best value: 0.129531:  10%|█         | 2/20 [00:16<02:40,  8.92s/it] 

[I 2023-08-31 19:26:46,855] Trial 1 finished with value: 0.12953071987638148 and parameters: {'n_estimators': 159, 'max_depth': 11, 'min_samples_split': 3, 'min_samples_leaf': 10}. Best is trial 1 with value: 0.12953071987638148.


Best trial: 2. Best value: 0.169236:  15%|█▌        | 3/20 [00:20<01:56,  6.86s/it]

...

[I 2023-08-31 19:27:51,632] Trial 17 finished with value: 0.20064496624004838 and parameters: {'n_estimators': 21, 'max_depth': 20, 'min_samples_split': 3, 'min_samples_leaf': 1}. Best is trial 17 with value: 0.20064496624004838.


Best trial: 17. Best value: 0.200645:  95%|█████████▌| 19/20 [01:24<00:02,  2.95s/it]

[I 2023-08-31 19:27:55,256] Trial 18 finished with value: 0.19029863280593906 and parameters: {'n_estimators': 22, 'max_depth': 20, 'min_samples_split': 4, 'min_samples_leaf': 2}. Best is trial 17 with value: 0.20064496624004838.


Best trial: 17. Best value: 0.200645: 100%|██████████| 20/20 [01:28<00:00,  4.41s/it]

[I 2023-08-31 19:27:58,948] Trial 19 finished with value: 0.1880479693641036 and parameters: {'n_estimators': 22, 'max_depth': 23, 'min_samples_split': 4, 'min_samples_leaf': 3}. Best is trial 17 with value: 0.20064496624004838.


Predicting sub grade only has an accuracy of 20%.

### Saving the model

In [ ]:
model = RandomForestClassifier(**study.best_params)

pickle.dump(model, open('./model/app/model_grade.pkl', 'wb'))

# 7. Predicting Interest Rate

## 7.1. Data Preparation

Splitting data into X and y.

In [18]:
X = df.drop(columns=['int_rate'])
y = df['int_rate']

Sampling down.

In [19]:
X, y = resample(X, y, n_samples=len(X) // 10, random_state=42)

Train, test split.

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42, stratify=y)

Saving feature names.

In [21]:
categorical_features = X.dtypes[df.dtypes != np.number].index
numerical_features = X.dtypes[df.dtypes == np.number].index

### Model Tuning

In [22]:
encoder = OneHotEncoder()

column_transformer = ColumnTransformer([
    ('cat', encoder, categorical_features)
])

def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 10, 200, log=True)
    max_depth = trial.suggest_int('max_depth', 2, 32)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)

    model = RandomForestRegressor(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        random_state=42
    )

    pipe = make_pipeline(column_transformer, TruncatedSVD(), model)

    pipe.fit(X_train, y_train)

    y_pred = pipe.predict(X_test)
    
    mae = mean_absolute_error(y_test, y_pred)

    return mae

In [23]:
study = optuna.create_study(direction='minimize')

study.optimize(objective, n_trials=20, show_progress_bar=True)

[I 2023-08-31 19:27:59,244] A new study created in memory with name: no-name-20d44a51-f27e-4f1a-a63c-9fdce6f4afc5
Best trial: 0. Best value: 2.06893:   5%|▌         | 1/20 [00:10<03:26, 10.88s/it]

[I 2023-08-31 19:28:10,122] Trial 0 finished with value: 2.0689343861707785 and parameters: {'n_estimators': 87, 'max_depth': 13, 'min_samples_split': 8, 'min_samples_leaf': 1}. Best is trial 0 with value: 2.0689343861707785.


Best trial: 1. Best value: 1.97179:  10%|█         | 2/20 [00:14<01:59,  6.64s/it]

[I 2023-08-31 19:28:13,802] Trial 1 finished with value: 1.9717850066323606 and parameters: {'n_estimators': 21, 'max_depth': 25, 'min_samples_split': 7, 'min_samples_leaf': 9}. Best is trial 1 with value: 1.9717850066323606.


Best trial: 1. Best value: 1.97179:  15%|█▌        | 3/20 [00:16<01:13,  4.32s/it]

...

[I 2023-08-31 19:29:33,342] Trial 17 finished with value: 1.8410143697228312 and parameters: {'n_estimators': 41, 'max_depth': 24, 'min_samples_split': 5, 'min_samples_leaf': 1}. Best is trial 17 with value: 1.8410143697228312.


Best trial: 17. Best value: 1.84101:  95%|█████████▌| 19/20 [01:43<00:06,  6.87s/it]

[I 2023-08-31 19:29:42,356] Trial 18 finished with value: 1.8509658558346282 and parameters: {'n_estimators': 52, 'max_depth': 24, 'min_samples_split': 5, 'min_samples_leaf': 2}. Best is trial 17 with value: 1.8410143697228312.


Best trial: 17. Best value: 1.84101: 100%|██████████| 20/20 [01:53<00:00,  5.65s/it]

[I 2023-08-31 19:29:52,281] Trial 19 finished with value: 1.908366453143178 and parameters: {'n_estimators': 66, 'max_depth': 18, 'min_samples_split': 4, 'min_samples_leaf': 4}. Best is trial 17 with value: 1.8410143697228312.


Tuned model has Mean Absolute Error of 1.8410143697228312.

### Saving the Model

In [24]:
model = RandomForestRegressor(**study.best_params)

pickle.dump(model, open('./model/app/model_int_rate.pkl', 'wb'))

***

# End of Part 4

What was done?

We created and saved three models:
* Model predicting loan grade
* Model predicting loan sub grade
* Model predicting interest rate